In [ ]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload

# import own functions
from utils import find_folders

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
##LOAD NEW FILES
subID = 'Sub021'

fft110_fname = subID + '_FFT_110.npy'
fft125_fname = subID + '_FFT_125.npy'
fft145_fname = subID + '_FFT_145.npy'

fnames_path = os.path.join(project_path,
    'data',
    'switching_freqs',
    subID)

fft110 = np.load(os.path.join(
    fnames_path,
    fft110_fname
))

fft125 = np.load(os.path.join(
    fnames_path,
    fft125_fname
))

fft145 = np.load(os.path.join(
    fnames_path,
    fft145_fname
))

In [ ]:
stim = np.full((1,16),3.4)

In [ ]:
### PLOT SPECTROGRAMS
%matplotlib qt

fig = plt.figure(figsize = (9,4))
ax = [plt.subplot(1,3,i+1) for i in range(3)]

for a in ax[1:3]:
    a.set_yticklabels([])
for a in ax:  a.set_xlabel('Time [sec]')
#plt.subplots_adjust(wspace = 1, hspace = 0)

tp1 = 18
tp2 = 787
tp3 = 188

ax1 = ax[0].twinx()
ax2 = ax[1].twinx()
ax3 = ax[2].twinx()

ax[0].pcolormesh(fft110[1,:,tp1:tp1+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax1.plot(np.arange(0,16), stim[0], color = 'white', linewidth = 2.5, linestyle = 'dotted')
ax1.set_yticks(np.arange(2,4.5,0.5), labels = [])
ax[0].set_ylabel('Frequency [Hz]')
ax[0].set_title('Stim 110Hz')

ax[1].pcolormesh(fft125[1,:,tp2:tp2+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax2.plot(np.arange(0,16), stim[0], color = 'white', linewidth = 2.5, linestyle = 'dotted')
ax2.set_yticks(np.arange(2,4.5,0.5), labels = [])
ax[1].set_title('Stim 125Hz')

ax[2].pcolormesh(fft145[1,:,tp3:tp3+15], cmap = 'viridis', vmin = 0, vmax = 0.2)
ax3.plot(np.arange(0,16), stim[0], color = 'white', linewidth = 2.5, linestyle = 'dotted')
ax3.set_yticks(np.arange(2,4.5,0.5), labels = [])
ax[2].set_title('Stim 145Hz')


for a in ax:
    a.set_ylim(40,100)

In [ ]:
plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig'
),
dpi = 400)

plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig.pdf'
))

In [ ]:
### PLOT POWER SPECTRA
tp1 = 16
tp2 = 948
tp3 = 262
%matplotlib qt 
fig, axes = plt.subplots(1,1, figsize = (4,4))

side = 1

ps_110 = np.mean(fft110[side,:,tp1:tp1+15],1)
ps_125 = np.mean(fft125[side,:,tp2:tp2+15],1)
ps_145 = np.mean(fft145[side,:,tp3:tp3+15],1)


plt.plot(np.arange(1,127),ps_110,label = 'Stim 110Hz', lw = 2, color = 'turquoise')
plt.plot(np.arange(1,127),ps_125,label = 'Stim 125Hz', lw = 2, color = 'midnightblue')
plt.plot(np.arange(1,127),ps_145,label = 'Stim 145Hz', lw = 2, color = 'deeppink')
    
plt.xlim(50,80)
plt.ylim(0,0.30)
plt.xlabel('Frequency [Hz]')
#plt.ylabel('LFP Power')
    
plt.legend()
plt.show()

In [ ]:
plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig_PS'
),
dpi = 200)

plt.savefig(os.path.join(
    fnames_path,
    subID + 'SubharmonicsFig_PS.pdf'
))

## Group Figure

In [ ]:
import glob

freqs_dir = '/Users//barbaramathiopoulou//Documents//PhD_worken//switching_freqs//'

freqs_of_interest = [110, 125, 145]

dict_times = [
    {'SubID': 'Sub021',
    'side': 1,
    'Onsets': [18, 787, 188]},

    {'SubID': 'Sub025',
    'side': 1,
    'Onsets': [1, 620, 146]},

    {'SubID': 'Sub028',
    'side': 1,
    'Onsets': [16, 948, 262]},

    {'SubID': 'Sub050',
    'side': 0,
    'Onsets': [115, 750, 120]}
]

In [ ]:
ps_dat_arrays = [[] for _ in range(3)]

for entry in dict_times:
    sub_id = entry['SubID']
    onsets = entry['Onsets']
    side = entry['side']

    folder_path = os.path.join(freqs_dir, sub_id)

    if os.path.exists(folder_path):
        # Iterate through files in the folder
        for i, onset in enumerate(onsets):
            # Find files ending with '_{onset}_freqs_of_interest.npy'
            file_pattern = os.path.join(folder_path, f"*_{freqs_of_interest[i]}.npy")
            files = glob.glob(file_pattern)
            if files:
                # Process the files as needed
                for file in files:
                    data = np.load(file)
                    ps_dat = np.mean(data[side,:,onset:onset+15],1)
                    ps_dat_arrays[i].append(ps_dat)

ps_dat_arrays = [np.array(ps_dat_list) for ps_dat_list in ps_dat_arrays]


In [ ]:
# Calculate mean and standard error of the mean (SEM)
mean_ps_dat = [np.mean(arr, axis=0) for arr in ps_dat_arrays]
sem_ps_dat = [np.std(arr, axis=0) / np.sqrt(arr.shape[0]) for arr in ps_dat_arrays]

# Plotting
plt.plot(np.arange(1, 127), mean_ps_dat[0], label='Stim 110Hz', lw=2, color='darkturquoise')
plt.plot(np.arange(1, 127), mean_ps_dat[1], label='Stim 125Hz', lw=2, color='midnightblue')
plt.plot(np.arange(1, 127), mean_ps_dat[2], label='Stim 145Hz', lw=2, color='deeppink')

# Plot shaded areas for standard error of the mean (SEM)
plt.fill_between(np.arange(1, 127), mean_ps_dat[0] - sem_ps_dat[0], mean_ps_dat[0] + sem_ps_dat[0], color='lightcyan', alpha=0.5)
plt.fill_between(np.arange(1, 127), mean_ps_dat[1] - sem_ps_dat[1], mean_ps_dat[1] + sem_ps_dat[1], color='lightblue', alpha=0.5)
plt.fill_between(np.arange(1, 127), mean_ps_dat[2] - sem_ps_dat[2], mean_ps_dat[2] + sem_ps_dat[2], color='lightpink', alpha=0.5)

plt.xlim(50,80)
plt.ylim(0, 0.20)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')
plt.legend()
plt.show()

In [ ]:
plt.savefig(os.path.join(freqs_dir,'Group_freqs'), dpi = 150)
plt.savefig(os.path.join(freqs_dir,'Group_freqs.pdf'))

# Individual Plots of An Signal Comparing Entrain Threshold

In [ ]:
freqs_amps = pd.read_excel(os.path.join(
    project_path,'results','Freq_Adjusting','Freqs_Amps.xlsx'
), sheet_name='Sheet1')

freqs_amps['SubCode'] = freqs_amps['SubID'].astype('category').cat.codes + 1
freqs_amps

In [ ]:
plt.figure(figsize=(8, 6))
for subid in freqs_amps['SubID'].unique():
    sub_df = freqs_amps[freqs_amps['SubID'] == subid]
    plt.plot(sub_df['Freq'], sub_df['Amplitude'], label=f'{subid}', marker = 'o')
    #plt.scatter(sub_df['Freq'], sub_df['Amplitude'], label=f'{subid}')

# Add labels and legend
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.legend()

# Show plot
plt.show()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Assuming freqs_amps is your DataFrame with 'Amplitude', 'Freq', and 'SubCode' columns

# Define and fit the linear mixed-effects model
model = sm.MixedLM.from_formula("Amplitude ~ Freq", data=freqs_amps, groups=freqs_amps["SubCode"])
result = model.fit()

# Print the summary of the fitted model
print(result.summary())
